In [46]:
import requests
import json
import sys
import pandas as pd
import csv
from  keys  import  client_id, api_key
print(sys.path)

['/Users/Mike/Flatiron_020121/yelp_phase1_project', '/opt/anaconda3/lib/python38.zip', '/opt/anaconda3/lib/python3.8', '/opt/anaconda3/lib/python3.8/lib-dynload', '', '/opt/anaconda3/lib/python3.8/site-packages', '/opt/anaconda3/lib/python3.8/site-packages/aeosa', '/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions', '/Users/Mike/.ipython']


In [111]:
url =  'https://api.yelp.com/v3/businesses/search' #points to url of user's yelp developer page

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }


# Dynamic: change term, location, and categories to pull different results from call
# * Required for defining url_params
term = 'Film Production'
location = 'Brooklyn'
categories = 'Video/Film'
spec = f'{term}_{location}_{categories}_data'

url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "limit": 50,
                "offset": 0
            }

"""
The variables and requests below are critical, 
but have been written in to yelp_call() fn
"""
# response = requests.get(url, headers=headers, params=url_params)
# data = response.json()

In [115]:
def yelp_call(headers, url_params):
    """
    This will use the requests module to get from Yelp. 
    What is returned will be modified by our URL parameters.
    This must be called fresh with updated url_params for each call if we want to return more results.
    
    2/14 — defined data variable in this loop.
    """
    response = requests.get(url, headers=headers, params=url_params) # our url, header and params should be consistent, atleast with our Yelp data
    data = response.json()
    return data

In [108]:
yelp_call(headers, url_params)

{'businesses': [{'id': 'CwBYC8nVLbG7KjVU6nwR4A',
   'alias': 'dijifi-brooklyn',
   'name': 'DiJiFi',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/rzH6f1btqYDEXZtrCa_uFg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/dijifi-brooklyn?adjust_creative=vGqQbkXzrgSl7z2qib6lAg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=vGqQbkXzrgSl7z2qib6lAg',
   'review_count': 165,
   'categories': [{'alias': 'photographystores',
     'title': 'Photography Stores & Services'},
    {'alias': 'videofilmproductions', 'title': 'Video/Film Production'},
    {'alias': 'digitizingservices', 'title': 'Digitizing Services'}],
   'rating': 4.5,
   'coordinates': {'latitude': 40.7382584, 'longitude': -73.9549479},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '1166 Manhattan Ave',
    'address2': 'Ste 201',
    'address3': '',
    'city': 'Brooklyn',
    'zip_code': '11222',
    'country': 'US',
    'state': 'NY',
    'display_address': 

In [117]:
# data['total']
yelp_call(headers, url_params)['total']

298

In [50]:
def parse_data(list_of_data):
    """
    Input data['businesses'] to return a list of tuples,
    with each tuple containing individual business name, address, and other items
    """
    businesses = []
    for business in list_of_data:
        biz_price = None
        if 'price' not in business.keys():
            biz_price = None
        else:
            biz_price = len(business['price'])
        biz_tuple = (business['name'],
                     business['location']['display_address'],
                     business['location']['city'],
                     business['rating'],
                     business['review_count'],
                     business['coordinates'],
                     biz_price)
        businesses.append(biz_tuple)
    return businesses

In [344]:
"""
JAKE I FUCKING DID IT

WE DON'T EVEN GET AN ERROR MESSAGE ANYMORE
"""
csv_filepath = f'database/{term}_{location}_database.csv'

def call_1000(csv_filepath):
    url_params['offset'] = 0
    results = yelp_call(headers, url_params)
    parsed = parse_data(results['businesses']) # list of businesses in tuples
    num = results['total']
    biz_list = []
    while url_params['offset'] < 1000 and len(biz_list) < num:
        for biz in parsed:
            biz_list.append(biz)
        url_params['offset'] += 50
        results = yelp_call(headers, url_params)
        if num >= len(biz_list):
            if 'businesses' not in results:
                break
            else:
                parsed = parse_data(results['businesses']) # list of businesses in tuples
        elif len(biz_list) <= 950:
            continue
        else:
            break
    df = pd.DataFrame(biz_list, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
    with open(csv_filepath, "a") as f: #'x' creates and writes to csv
        read_file = csv.writer(f)
        df.to_csv(csv_filepath, mode = "a", index = False)
    print('CSV file written to {csv_filepath}.')
    return df


In [345]:
call_1000(csv_filepath)

,Name,Address,City,Rating,Review Count,Coordinates,Price
0,DiJiFi,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,4.5,165,"{'latitude': 40.7382584, 'longitude': -73.9549...",2.0
1,590films,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,5.0,4,"{'latitude': 40.7280427, 'longitude': -73.9588...",NaN
2,Kaleidoscope Pictures,"[Brooklyn, NY 11231]",Brooklyn,5.0,8,"{'latitude': 40.6772802, 'longitude': -74.0094...",NaN
3,Shell's Loft Brooklyn,"[13 Lexington Ave, Brooklyn, NY 11238]",Brooklyn,5.0,2,"{'latitude': 40.6866128, 'longitude': -73.9616...",NaN
4,Mind of Who Productions,"[193 Marcus Garvey Blvd, Brooklyn, NY 11221]",Brooklyn,4.5,2,"{'latitude': 40.691765, 'longitude': -73.939434}",NaN
...,...,...,...,...,...,...,...
291,Secret Fire Media,"[Wappingers Falls, NY 12590]",Wappingers Falls,4.0,10,"{'latitude': 41.59696, 'longitude': -73.91325}",NaN
292,Cinescope,"[5 S Main St, Doylestown, PA 18901]",Doylestown,5.0,3,"{'latitude': 40.309828966856, 'longitude': -75...",NaN
293,Jordan Green Productions,"[3502 Leavenworth St, Omaha, NE 68105]",Omaha,1.0,1,"{'latitude': 41.252651, 'longitude': -95.964111}",NaN
294,BeauMonde Originals,"[3128 N 35th St, Philadelphia, PA 19132]",Philadelphia,5.0,2,"{'latitude': 40.00559, 'longitude': -75.18678}",NaN


<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

# Tests below 
### (first cell through yelp call must be run before these)

In [73]:
parsed = parse_data(data['businesses'])
parsed

[('DiJiFi',
  ['1166 Manhattan Ave', 'Ste 201', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  165,
  {'latitude': 40.7382584, 'longitude': -73.9549479},
  2),
 ('590films',
  ['42 West St', 'Ste 301', 'Brooklyn, NY 11222'],
  'Brooklyn',
  5.0,
  4,
  {'latitude': 40.7280427, 'longitude': -73.9588363},
  None),
 ("Shell's Loft Brooklyn",
  ['13 Lexington Ave', 'Brooklyn, NY 11238'],
  'Brooklyn',
  5.0,
  2,
  {'latitude': 40.6866128, 'longitude': -73.9616521},
  None),
 ('Mind of Who Productions',
  ['193 Marcus Garvey Blvd', 'Brooklyn, NY 11221'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.691765, 'longitude': -73.939434},
  None),
 ('Kaleidoscope Pictures',
  ['Brooklyn, NY 11231'],
  'Brooklyn',
  5.0,
  8,
  {'latitude': 40.6772802, 'longitude': -74.0094471},
  None),
 ('Cinema World Studios',
  ['220 Dupont St', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.735954, 'longitude': -73.950508},
  None),
 ('Starr Street Media',
  ['852 Rogers Ave', 'Brooklyn, NY 

In [74]:
pd.DataFrame(parsed, columns =['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])

,Name,Address,City,Rating,Review Count,Coordinates,Price
0,DiJiFi,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,4.5,165,"{'latitude': 40.7382584, 'longitude': -73.9549...",2.0
1,590films,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,5.0,4,"{'latitude': 40.7280427, 'longitude': -73.9588...",NaN
2,Shell's Loft Brooklyn,"[13 Lexington Ave, Brooklyn, NY 11238]",Brooklyn,5.0,2,"{'latitude': 40.6866128, 'longitude': -73.9616...",NaN
3,Mind of Who Productions,"[193 Marcus Garvey Blvd, Brooklyn, NY 11221]",Brooklyn,4.5,2,"{'latitude': 40.691765, 'longitude': -73.939434}",NaN
4,Kaleidoscope Pictures,"[Brooklyn, NY 11231]",Brooklyn,5.0,8,"{'latitude': 40.6772802, 'longitude': -74.0094...",NaN
5,Cinema World Studios,"[220 Dupont St, Brooklyn, NY 11222]",Brooklyn,4.5,2,"{'latitude': 40.735954, 'longitude': -73.950508}",NaN
6,Starr Street Media,"[852 Rogers Ave, Brooklyn, NY 11226]",Brooklyn,5.0,1,"{'latitude': 40.64986, 'longitude': -73.95257}",NaN
7,Hit and Run Productions,"[65 Roebling St, Ste 202, Brooklyn, NY 11211]",Brooklyn,5.0,4,"{'latitude': 40.7165128, 'longitude': -73.9546...",NaN
8,BK Actors Self-Tape,"[141 Flushing Ave, Bldg 77 ,Ste 1336, Brooklyn...",Brooklyn,5.0,14,"{'latitude': 40.6985551, 'longitude': -73.9712...",NaN
9,Bizvidmedia Productions,"[99 Wallstreet, New York, NY 10005]",New York,5.0,5,"{'latitude': 40.70479, 'longitude': -74.00726}",NaN


In [53]:
parsed

[('DiJiFi',
  ['1166 Manhattan Ave', 'Ste 201', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  165,
  {'latitude': 40.7382584, 'longitude': -73.9549479},
  2),
 ('590films',
  ['42 West St', 'Ste 301', 'Brooklyn, NY 11222'],
  'Brooklyn',
  5.0,
  4,
  {'latitude': 40.7280427, 'longitude': -73.9588363},
  None),
 ("Shell's Loft Brooklyn",
  ['13 Lexington Ave', 'Brooklyn, NY 11238'],
  'Brooklyn',
  5.0,
  2,
  {'latitude': 40.6866128, 'longitude': -73.9616521},
  None),
 ('Mind of Who Productions',
  ['193 Marcus Garvey Blvd', 'Brooklyn, NY 11221'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.691765, 'longitude': -73.939434},
  None),
 ('Kaleidoscope Pictures',
  ['Brooklyn, NY 11231'],
  'Brooklyn',
  5.0,
  8,
  {'latitude': 40.6772802, 'longitude': -74.0094471},
  None),
 ('Cinema World Studios',
  ['220 Dupont St', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.735954, 'longitude': -73.950508},
  None),
 ('Starr Street Media',
  ['852 Rogers Ave', 'Brooklyn, NY 

In [54]:
type(len(df))

int

In [72]:
df = pd.DataFrame(data, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
results = yelp_call(headers, url_params)
parsed = parse_data(results['businesses'])
df_2 = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
df.append(df_2, ignore_index=True)
df

,Name,Address,City,Rating,Review Count,Coordinates,Price


### Testing for call_1000

In [72]:
df = pd.DataFrame(data, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
results = yelp_call(headers, url_params)
parsed = parse_data(results['businesses'])
df_2 = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
df.append(df_2, ignore_index=True)
df

,Name,Address,City,Rating,Review Count,Coordinates,Price


In [ ]:
csv_filepath = f'database/{term}_{location}_database.csv'

In [ ]:
current_offset = 0
url_params['offset'] = current_offset
results = yelp_call(headers, url_params)
parsed = parse_data(results['businesses'])
num = results['total']
df = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
while current_offset < (num+50) and current_offset < 1000:     

    
#         concat or append df here
#             current_offset += 50
#             url_params['offset'] = current_offset
#             results = yelp_call(headers, url_params)
#             parsed = parse_data(results['businesses'])

        df2 = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
        df.append(df2)
    current_offset += 50
    url_params['offset'] = current_offset
    results = yelp_call(headers, url_params)
    parsed = parse_data(results['businesses'])
return df

In [71]:
#testing call_1000() contents outside of function

current_offset = url_params['offset']
df = pd.DataFrame(data, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
num = 0
while current_offset < (num+1) and current_offset < 951: 
    url_params['offset'] = current_offset

    results = yelp_call(headers, url_params)
    parsed = parse_data(results['businesses'])
    #concat or append df here
    df_2 = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
    df.append(df_2, ignore_index=True)
#     num += len(df)
#     current_offset += 50
#     url_params['offset'] = current_offset
    print(df, current_offset, num, url_params)

Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categori

KeyboardInterrupt: 

In [29]:
pd.DataFrame(data)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [103]:
# testing DataFrame output from parsed (parsed = parse_data(data['businesses']))

pd.DataFrame(parsed[0:3], columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])

,Name,Address,City,Rating,Review Count,Coordinates,Price
0,DiJiFi,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,4.5,165,"{'latitude': 40.7382584, 'longitude': -73.9549...",2.0
1,590films,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,5.0,4,"{'latitude': 40.7280427, 'longitude': -73.9588...",NaN
2,Shell's Loft Brooklyn,"[13 Lexington Ave, Brooklyn, NY 11238]",Brooklyn,5.0,2,"{'latitude': 40.6866128, 'longitude': -73.9616...",NaN


In [117]:
# data['total']
yelp_call(headers, url_params)['total']

298

In [ ]:
# df = pd.DataFrame(parsed_results, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])

In [ ]:
# your code to open the csv file, concat the current data, and save the data. 

def df_save(csv_filepath, df): # MIKE NOTE: an if statement for if the file exists may be useful
    with open(csv_filepath, "a") as f: #'x' creates and writes to csv
        read_file = csv.writer(f)
        df.to_csv(csv_filepath, mode = "a", index = False)
# this will append to csv. 2nd arg index=False to not save the inde

In [ ]:
df_save(f'database/{term}_{location}_database.csv', df)

In [71]:
current_offset = url_params['offset']
df = pd.DataFrame(data, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
num = 0
while current_offset < (num+1) and current_offset < 951: 
    url_params['offset'] = current_offset

    results = yelp_call(headers, url_params)
    parsed = parse_data(results['businesses'])
    #concat or append df here
    df_2 = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
    df.append(df_2, ignore_index=True)
#     num += len(df)
#     current_offset += 50
#     url_params['offset'] = current_offset
    print(df, current_offset, num, url_params)

Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categories': 'Video/Film', 'limit': 50, 'offset': 0}
Empty DataFrame
Columns: [Name, Address, City, Rating, Review Count, Coordinates, Price]
Index: [] 0 0 {'term': 'Film+Production', 'location': 'Brooklyn', 'categori

KeyboardInterrupt: 

In [73]:
parsed = parse_data(data['businesses'])
parsed

[('DiJiFi',
  ['1166 Manhattan Ave', 'Ste 201', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  165,
  {'latitude': 40.7382584, 'longitude': -73.9549479},
  2),
 ('590films',
  ['42 West St', 'Ste 301', 'Brooklyn, NY 11222'],
  'Brooklyn',
  5.0,
  4,
  {'latitude': 40.7280427, 'longitude': -73.9588363},
  None),
 ("Shell's Loft Brooklyn",
  ['13 Lexington Ave', 'Brooklyn, NY 11238'],
  'Brooklyn',
  5.0,
  2,
  {'latitude': 40.6866128, 'longitude': -73.9616521},
  None),
 ('Mind of Who Productions',
  ['193 Marcus Garvey Blvd', 'Brooklyn, NY 11221'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.691765, 'longitude': -73.939434},
  None),
 ('Kaleidoscope Pictures',
  ['Brooklyn, NY 11231'],
  'Brooklyn',
  5.0,
  8,
  {'latitude': 40.6772802, 'longitude': -74.0094471},
  None),
 ('Cinema World Studios',
  ['220 Dupont St', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.735954, 'longitude': -73.950508},
  None),
 ('Starr Street Media',
  ['852 Rogers Ave', 'Brooklyn, NY 

In [74]:
pd.DataFrame(parsed, columns =['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])

,Name,Address,City,Rating,Review Count,Coordinates,Price
0,DiJiFi,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,4.5,165,"{'latitude': 40.7382584, 'longitude': -73.9549...",2.0
1,590films,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,5.0,4,"{'latitude': 40.7280427, 'longitude': -73.9588...",NaN
2,Shell's Loft Brooklyn,"[13 Lexington Ave, Brooklyn, NY 11238]",Brooklyn,5.0,2,"{'latitude': 40.6866128, 'longitude': -73.9616...",NaN
3,Mind of Who Productions,"[193 Marcus Garvey Blvd, Brooklyn, NY 11221]",Brooklyn,4.5,2,"{'latitude': 40.691765, 'longitude': -73.939434}",NaN
4,Kaleidoscope Pictures,"[Brooklyn, NY 11231]",Brooklyn,5.0,8,"{'latitude': 40.6772802, 'longitude': -74.0094...",NaN
5,Cinema World Studios,"[220 Dupont St, Brooklyn, NY 11222]",Brooklyn,4.5,2,"{'latitude': 40.735954, 'longitude': -73.950508}",NaN
6,Starr Street Media,"[852 Rogers Ave, Brooklyn, NY 11226]",Brooklyn,5.0,1,"{'latitude': 40.64986, 'longitude': -73.95257}",NaN
7,Hit and Run Productions,"[65 Roebling St, Ste 202, Brooklyn, NY 11211]",Brooklyn,5.0,4,"{'latitude': 40.7165128, 'longitude': -73.9546...",NaN
8,BK Actors Self-Tape,"[141 Flushing Ave, Bldg 77 ,Ste 1336, Brooklyn...",Brooklyn,5.0,14,"{'latitude': 40.6985551, 'longitude': -73.9712...",NaN
9,Bizvidmedia Productions,"[99 Wallstreet, New York, NY 10005]",New York,5.0,5,"{'latitude': 40.70479, 'longitude': -74.00726}",NaN


In [53]:
parsed

[('DiJiFi',
  ['1166 Manhattan Ave', 'Ste 201', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  165,
  {'latitude': 40.7382584, 'longitude': -73.9549479},
  2),
 ('590films',
  ['42 West St', 'Ste 301', 'Brooklyn, NY 11222'],
  'Brooklyn',
  5.0,
  4,
  {'latitude': 40.7280427, 'longitude': -73.9588363},
  None),
 ("Shell's Loft Brooklyn",
  ['13 Lexington Ave', 'Brooklyn, NY 11238'],
  'Brooklyn',
  5.0,
  2,
  {'latitude': 40.6866128, 'longitude': -73.9616521},
  None),
 ('Mind of Who Productions',
  ['193 Marcus Garvey Blvd', 'Brooklyn, NY 11221'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.691765, 'longitude': -73.939434},
  None),
 ('Kaleidoscope Pictures',
  ['Brooklyn, NY 11231'],
  'Brooklyn',
  5.0,
  8,
  {'latitude': 40.6772802, 'longitude': -74.0094471},
  None),
 ('Cinema World Studios',
  ['220 Dupont St', 'Brooklyn, NY 11222'],
  'Brooklyn',
  4.5,
  2,
  {'latitude': 40.735954, 'longitude': -73.950508},
  None),
 ('Starr Street Media',
  ['852 Rogers Ave', 'Brooklyn, NY 

In [54]:
type(len(df))

int

In [204]:
called_1k = call_1000()

In [127]:
parsed
test_d = pd.DataFrame
test_d2 = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
pd.concat([test_d, parsed])
test_d
type(parsed)

TypeError: _consolidate() missing 1 required positional argument: 'self'

In [205]:
called_1k.iloc[45:60]

,Name,Address,City,Rating,Review Count,Coordinates,Price
45,Brain Fry Productions,"[2853 Steinway St, 2F, Astoria, NY 11103]",Astoria,5.0,2,"{'latitude': 40.630508, 'longitude': -74.028456}",NaN
46,Silvercup Studios,"[42-22 22nd St, Long Island City, NY 11101]",Long Island City,4.5,12,"{'latitude': 40.751331, 'longitude': -73.943515}",NaN
47,YouTube Space,"[75 9th Ave, New York, NY 10011]",New York,5.0,6,"{'latitude': 40.742581, 'longitude': -74.005434}",NaN
48,Sun Production,"[36-06 30th St, C7, Astoria, NY 11106]",Astoria,5.0,1,"{'latitude': 40.756815, 'longitude': -73.930988}",NaN
49,Broadway Stages,"[359 Kingsland Ave, Greenpoint, NY 11222]",Greenpoint,4.0,1,"{'latitude': 40.7289565103425, 'longitude': -7...",NaN


In [163]:
test_parsed = parse_data(results['businesses'])
test_parsed
test_d = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
test_d.append(parsed)

,0,1,2,3,4,5,6,Address,City,Coordinates,Name,Price,Rating,Review Count
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,"{'latitude': 40.7382584, 'longitude': -73.9549...",DiJiFi,2.0,4.5,165.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,"{'latitude': 40.7280427, 'longitude': -73.9588...",590films,NaN,5.0,4.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[13 Lexington Ave, Brooklyn, NY 11238]",Brooklyn,"{'latitude': 40.6866128, 'longitude': -73.9616...",Shell's Loft Brooklyn,NaN,5.0,2.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[193 Marcus Garvey Blvd, Brooklyn, NY 11221]",Brooklyn,"{'latitude': 40.691765, 'longitude': -73.939434}",Mind of Who Productions,NaN,4.5,2.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Brooklyn, NY 11231]",Brooklyn,"{'latitude': 40.6772802, 'longitude': -74.0094...",Kaleidoscope Pictures,NaN,5.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,Brain Fry Productions,"[2853 Steinway St, 2F, Astoria, NY 11103]",Astoria,5.0,2.0,"{'latitude': 40.630508, 'longitude': -74.028456}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,Silvercup Studios,"[42-22 22nd St, Long Island City, NY 11101]",Long Island City,4.5,12.0,"{'latitude': 40.751331, 'longitude': -73.943515}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,YouTube Space,"[75 9th Ave, New York, NY 10011]",New York,5.0,6.0,"{'latitude': 40.742581, 'longitude': -74.005434}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,Sun Production,"[36-06 30th St, C7, Astoria, NY 11106]",Astoria,5.0,1.0,"{'latitude': 40.756815, 'longitude': -73.930988}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
df = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
df

,Name,Address,City,Rating,Review Count,Coordinates,Price
0,DiJiFi,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,4.5,165,"{'latitude': 40.7382584, 'longitude': -73.9549...",2.0
1,590films,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,5.0,4,"{'latitude': 40.7280427, 'longitude': -73.9588...",NaN
2,Shell's Loft Brooklyn,"[13 Lexington Ave, Brooklyn, NY 11238]",Brooklyn,5.0,2,"{'latitude': 40.6866128, 'longitude': -73.9616...",NaN
3,Mind of Who Productions,"[193 Marcus Garvey Blvd, Brooklyn, NY 11221]",Brooklyn,4.5,2,"{'latitude': 40.691765, 'longitude': -73.939434}",NaN
4,Kaleidoscope Pictures,"[Brooklyn, NY 11231]",Brooklyn,5.0,8,"{'latitude': 40.6772802, 'longitude': -74.0094...",NaN
5,Cinema World Studios,"[220 Dupont St, Brooklyn, NY 11222]",Brooklyn,4.5,2,"{'latitude': 40.735954, 'longitude': -73.950508}",NaN
6,Starr Street Media,"[852 Rogers Ave, Brooklyn, NY 11226]",Brooklyn,5.0,1,"{'latitude': 40.64986, 'longitude': -73.95257}",NaN
7,Hit and Run Productions,"[65 Roebling St, Ste 202, Brooklyn, NY 11211]",Brooklyn,5.0,4,"{'latitude': 40.7165128, 'longitude': -73.9546...",NaN
8,BK Actors Self-Tape,"[141 Flushing Ave, Bldg 77 ,Ste 1336, Brooklyn...",Brooklyn,5.0,14,"{'latitude': 40.6985551, 'longitude': -73.9712...",NaN
9,Bizvidmedia Productions,"[99 Wallstreet, New York, NY 10005]",New York,5.0,5,"{'latitude': 40.70479, 'longitude': -74.00726}",NaN


In [206]:
called_1k.tail()

,Name,Address,City,Rating,Review Count,Coordinates,Price
45,Brain Fry Productions,"[2853 Steinway St, 2F, Astoria, NY 11103]",Astoria,5.0,2,"{'latitude': 40.630508, 'longitude': -74.028456}",NaN
46,Silvercup Studios,"[42-22 22nd St, Long Island City, NY 11101]",Long Island City,4.5,12,"{'latitude': 40.751331, 'longitude': -73.943515}",NaN
47,YouTube Space,"[75 9th Ave, New York, NY 10011]",New York,5.0,6,"{'latitude': 40.742581, 'longitude': -74.005434}",NaN
48,Sun Production,"[36-06 30th St, C7, Astoria, NY 11106]",Astoria,5.0,1,"{'latitude': 40.756815, 'longitude': -73.930988}",NaN
49,Broadway Stages,"[359 Kingsland Ave, Greenpoint, NY 11222]",Greenpoint,4.0,1,"{'latitude': 40.7289565103425, 'longitude': -7...",NaN


In [163]:
test_parsed = parse_data(results['businesses'])
test_parsed
test_d = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
test_d.append(parsed)

,0,1,2,3,4,5,6,Address,City,Coordinates,Name,Price,Rating,Review Count
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,"{'latitude': 40.7382584, 'longitude': -73.9549...",DiJiFi,2.0,4.5,165.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,"{'latitude': 40.7280427, 'longitude': -73.9588...",590films,NaN,5.0,4.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[13 Lexington Ave, Brooklyn, NY 11238]",Brooklyn,"{'latitude': 40.6866128, 'longitude': -73.9616...",Shell's Loft Brooklyn,NaN,5.0,2.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[193 Marcus Garvey Blvd, Brooklyn, NY 11221]",Brooklyn,"{'latitude': 40.691765, 'longitude': -73.939434}",Mind of Who Productions,NaN,4.5,2.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Brooklyn, NY 11231]",Brooklyn,"{'latitude': 40.6772802, 'longitude': -74.0094...",Kaleidoscope Pictures,NaN,5.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,Brain Fry Productions,"[2853 Steinway St, 2F, Astoria, NY 11103]",Astoria,5.0,2.0,"{'latitude': 40.630508, 'longitude': -74.028456}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,Silvercup Studios,"[42-22 22nd St, Long Island City, NY 11101]",Long Island City,4.5,12.0,"{'latitude': 40.751331, 'longitude': -73.943515}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,YouTube Space,"[75 9th Ave, New York, NY 10011]",New York,5.0,6.0,"{'latitude': 40.742581, 'longitude': -74.005434}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,Sun Production,"[36-06 30th St, C7, Astoria, NY 11106]",Astoria,5.0,1.0,"{'latitude': 40.756815, 'longitude': -73.930988}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [336]:
len(biz_list)

296

In [337]:
num
# len(biz_list)

298

In [341]:
df.iloc[198:202]

,Name,Address,City,Rating,Review Count,Coordinates,Price
198,Musiclab NYC,"[Elmont, NY 11003]",Elmont,5.0,1,"{'latitude': 40.6943581912167, 'longitude': -7...",NaN
199,Crystal Lake Pictures,"[101 Crawfords Corner Rd, Ste 4-101R, Holmdel,...",Holmdel,3.0,2,"{'latitude': 40.3650877, 'longitude': -74.167457}",NaN
200,Dancejock Productions,"[40 Burt Dr, Ste 1, Deer Park, NY 11729]",Deer Park,4.5,12,"{'latitude': 40.758861, 'longitude': -73.305488}",NaN
201,rjephoto,"[basking ridge, NJ 07920]",basking ridge,5.0,2,"{'latitude': 40.7064399719238, 'longitude': -7...",NaN


In [288]:
url_params

{'term': 'Film+Production',
 'location': 'Brooklyn',
 'categories': 'Video/Film',
 'limit': 50,
 'offset': 0}

In [246]:
"""
GOD FUCKING DAMMIT
"""
df.iloc[48:52]

,Name,Address,City,Rating,Review Count,Coordinates,Price
48,Steve Palmer Films,"[New York, NY 11215]",New York,5.0,1,"{'latitude': 40.66874, 'longitude': -73.98181}",NaN
49,Prosper Digital TV,"[33 35th St, Fl 5. Ste 523, Brooklyn, NY 11232]",Brooklyn,5.0,6,"{'latitude': 40.65722, 'longitude': -74.00735}",NaN
50,DiJiFi,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,4.5,165,"{'latitude': 40.7382584, 'longitude': -73.9549...",2.0
51,590films,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,5.0,4,"{'latitude': 40.7280427, 'longitude': -73.9588...",NaN


In [241]:
# len(biz_list) ## 1000

TypeError: unhashable type: 'list'

In [228]:
yelp_call(headers, url_params)

{'error': {'code': 'VALIDATION_ERROR',
  'description': 'Too many results requested, limit+offset must be <= 1000.'}}

In [231]:
len(biz_list)

0

In [ ]:
def get_restaurants(city, offset):
    """
    THIS IS ANOTHER APPROACH FROM GITHUB
    Make API request to Yelp to get restaurants for a city, and offset the results by an amount.

    Note that Yelp only returns 20 results each time, which is why we need to offset if we want
    the next Nth results.
    """

      """ WE HAVE THIS """
#     # Read Yelp API keys
#     with io.open('config_secret.json') as cred:
#         creds = json.load(cred)
#         auth = Oauth1Authenticator(**creds)
#         client = Client(auth)

    # Set search parameters for Yelp API request
    # Set term as restaurant to get restaurants back as the results
    # Also pass in offset, so Yelp knows how much to offset by

      """ WE HAVE THIS TOO """
    #     params = {
#         'term': 'restaurant',
#         'offset': offset
#     }

    # Make Yelp API call and return the API response
    return client.search(city, **params)

In [113]:
results = yelp_call(headers, url_params)
parsed = parse_data(results['businesses'])
len(parsed)

50

In [ ]:
 """ 
 THIS IS PART 2 OF ANOTHER APPROACH FROM GITHUB
 Get all restaurants for a city from Yelp and load restaurants into database.
 """

    # Get city id, as city id is a required parameter when adding a restaurant to the database
    city_id = get_city_id(city)

    # Start offset at 0 to return the first 20 results from Yelp API request
    offset = 0
    response = get_restaurants(city, offset)

    # Get total number of restaurants for this city
    total_results = response.total 
    """ 
    ^ AHA, THIS IS WHAT WE NEED FOR num IN OUR WHILE LOOP!
    
    JAKE POINTED TO THIS BUT INPUT TOTAL OF DATA VARIABLE, NOT JSON TOTAL
    
    WOULD BE EQUIVALENT TO LEN(PARSED) RIGHT AFTER DEFINING PARSED IN OUR WHILE LOOP
    
    PROBABLY DO NOT NEED IT
    ACTUALLY ENSURING THAT WE ARE NOT CALLING MORE THAN TOTAL POSSIBLE RESULTS 
    (AS JAKE WAS DOING WITH DATA['TOTAL'] IS SMART
    """

    # Offset by 20 each time to get all restaurants and load each restaurant into the database
    while offset < total_results:

        #CODE BELOW is essentially our PARSE_DATA() function
        for business in response.businesses:
            restaurant = Restaurant(city_id=city_id,
                                    name=business.name,
                                    address=" ".join(business.location.display_address),
                                    phone=business.display_phone,
                                    image_url=business.image_url,
                                    latitude=business.location.coordinate.latitude,
                                    longitude=business.location.coordinate.longitude)

            # Add each restaurant to the db
        #THIS IS WHAT WE NEED TO DO CORRECTLY
            db.session.add(restaurant)

        offset += 20

        response = get_restaurants(city, offset)


In [103]:
pd.DataFrame(parsed[0:3], columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])

,Name,Address,City,Rating,Review Count,Coordinates,Price
0,DiJiFi,"[1166 Manhattan Ave, Ste 201, Brooklyn, NY 11222]",Brooklyn,4.5,165,"{'latitude': 40.7382584, 'longitude': -73.9549...",2.0
1,590films,"[42 West St, Ste 301, Brooklyn, NY 11222]",Brooklyn,5.0,4,"{'latitude': 40.7280427, 'longitude': -73.9588...",NaN
2,Shell's Loft Brooklyn,"[13 Lexington Ave, Brooklyn, NY 11238]",Brooklyn,5.0,2,"{'latitude': 40.6866128, 'longitude': -73.9616...",NaN


In [203]:
def call_1000():
    """ MIKE NOTES: 
    12/13 — I believe num should be referring to the number of 
    results that are in our dataframe or dict.
    
    12/14 — Might not be a bad idea to loop through offset BEFORE writing to a csv file.
    This could be done by offsetting, adding new to a dict, then writing to a csv file
    *afterwards outside of our while-loop*. 
    ***Persistent opening/writing to csv is messy AF***
    
    df might need to be defined BEFORE while-loop, then APPENDED or CONCAT within while-loop.
    """
    current_offset = 0
    url_params['offset'] = current_offset
    results = yelp_call(headers, url_params)
    parsed = parse_data(results['businesses'])
    num = results['total']
#     total_called = len(parsed)
    df = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
    while current_offset < (num+50) and current_offset < 1000:     
#         concat or append df here
#             current_offset += 50
#             url_params['offset'] = current_offset
#             results = yelp_call(headers, url_params)
#             parsed = parse_data(results['businesses'])
            
            df2 = pd.DataFrame(parsed, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])
            df.append(df2)
        current_offset += 50
        url_params['offset'] = current_offset
        results = yelp_call(headers, url_params)
        parsed = parse_data(results['businesses'])
    return df


#     pd.concat([df, df_2])

# df_save(csv_filepath, df)
# pd.read_csv(csv_filepath)
# Jake's suggestion, create workaround that checks to see —— 
#if ['businesses'] in data.keys():